In [168]:
import pandas as pd
from influxdb import InfluxDBClient
import time
import json
import re
from datetime import datetime
from datetime import timezone
import requests
import os


# About 
this notebook takes vm data for a given flaky test
40 samples of fail are take, for each fail, data of around + 60 -60 s around is gather from VM 
since each VM has 11 measurements, they are all stored in separate csv files 
similarly 40 samples of passed sceario are take, again with +/ 60s time margin from VM 11 measruements. 


In [169]:
#optional, jupyter notebook dislay options to see more data
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None
pd.options.display.precision = 15

In [170]:
coin_auth = json.load(open('coin_credentials'))
coin_client = InfluxDBClient(
    host='testresults.qt.io', 
    username=coin_auth['username'],
    password=coin_auth['password'],
    port=8086,
    database='coin')

In [171]:
#checked_test_case = "qt/qtbase+tst_qwidget+enterLeaveOnWindowShowHide+Windows_11_22H2+X86_64+MSVC2019"
checked_test_case = "qt/qtwebengine+tst_qwebenginepage+backgroundColor+Windows_10_22H2+X86_64+MSVC2019"
res = checked_test_case.split("+")
project = res[0]
test_case = res[1]
test_function = res[2]
target_os_version=res[3]
target_arch=res[4]
target_compiler=res[5]
res

['qt/qtwebengine',
 'tst_qwebenginepage',
 'backgroundColor',
 'Windows_10_22H2',
 'X86_64',
 'MSVC2019']

In [172]:
coin_auth = json.load(open('coin_credentials'))
coin_client = InfluxDBClient(
    host='testresults.qt.io', 
    username=coin_auth['username'],
    password=coin_auth['password'],
    port=8086,
    database='coin')

In [173]:
q = "select * from flaky_test where branch = 'dev' and project = '{}' and testCase = '{}' and testFunction = '{}'\
and target_os_version = '{}' and target_arch = '{}' and target_compiler = '{}' and time > now() - 10d".format(project, test_case, test_function,\
                                                                                                            target_os_version, target_arch, target_compiler)
df = pd.DataFrame(coin_client.query(q).get_points())
df['platform'] = df['target_os_version'] + '+' + df['target_arch']+  '+' + df['target_compiler']
df['full_test_params'] = df['project'] + '+'+ df['testCase'] + '+' + df['testFunction'] +'+' + df['platform']
print(df.shape)
df.head()

(79, 28)


,time,branch,computeHost,cpu_count,cpu_count_1,cpu_seconds,duration,host_arch,host_compiler,host_os,host_os_version,id,path,platform_id,project,result,sha1,state,target_arch,target_compiler,target_os,target_os_version,taskType,testCase,testFunction,type,platform,full_test_params
0,2023-01-09T19:06:41.638051Z,dev,divine-oyster.on1.qt.io,4,None,7160,1790,X86_64,MSVC2019,Windows,Windows_10_22H2,1681269722,qt/qtwebengine/e26911562884ec6e72914da54065b0d67b96e516/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-e8e7c8Conan_DebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/55280872154174136dc6e0c716a4084e0ca5dc5e/test_1681270641,Windows10_22H2-MSVC2019-x64-host,qt/qtwebengine,Failed,a4342183199ba9fbd6c86939cf9748e069a09f9c,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,Integration,tst_qwebenginepage,backgroundColor,Test,Windows_10_22H2+X86_64+MSVC2019,qt/qtwebengine+tst_qwebenginepage+backgroundColor+Windows_10_22H2+X86_64+MSVC2019
1,2023-01-09T19:06:41.638051Z,dev,saved-toad.on1.qt.io,4,None,6972,1743,X86_64,MSVC2019,Windows,Windows_10_22H2,1681269722,qt/qtwebengine/e26911562884ec6e72914da54065b0d67b96e516/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-e8e7c8DebugAndRelease_Sccache_UseConfigure/0cdfe31876aa79c9baeb04ba306bb1e719a34b04/test_1681270633,None,qt/qtwebengine,Failed,a4342183199ba9fbd6c86939cf9748e069a09f9c,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,Integration,tst_qwebenginepage,backgroundColor,Test,Windows_10_22H2+X86_64+MSVC2019,qt/qtwebengine+tst_qwebenginepage+backgroundColor+Windows_10_22H2+X86_64+MSVC2019
2,2023-01-09T19:09:40.295973Z,dev,calm-squid.on1.qt.io,4,None,5896,1474,X86_64,MSVC2019,Windows,Windows_10_22H2,1681269726,qt/qtwebengine/a2ae7142f25ce37d8df09ae50e4c7544ea8840af/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-e8e7c8Conan_DebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/55280872154174136dc6e0c716a4084e0ca5dc5e/test_1681256652,Windows10_22H2-MSVC2019-x64-host,qt/qtwebengine,Failed,5836832dfde5687dae5762509e560e1c7f2258ce,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,Integration,tst_qwebenginepage,backgroundColor,Test,Windows_10_22H2+X86_64+MSVC2019,qt/qtwebengine+tst_qwebenginepage+backgroundColor+Windows_10_22H2+X86_64+MSVC2019
3,2023-01-10T00:07:11.695223Z,dev,dear-midge.on1.qt.io,4,None,6680,1670,X86_64,MSVC2019,Windows,Windows_10_22H2,1681293218,qt/qtwebengine/3e2697ec7e9d92cccfb27334bf92b2674b6c4c5a/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-e8e7c8Conan_DebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/822b699e52c68904b490b8097d0f0da0bd395039/test_1681293414,Windows10_22H2-MSVC2019-x64-host,qt/qtwebengine,Failed,c105706beae51fd785275206061b9fe1303658ed,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,Integration,tst_qwebenginepage,backgroundColor,Test,Windows_10_22H2+X86_64+MSVC2019,qt/qtwebengine+tst_qwebenginepage+backgroundColor+Windows_10_22H2+X86_64+MSVC2019
4,2023-01-10T03:41:49.174286Z,dev,big-grouse.on1.qt.io,4,None,6392,1598,X86_64,MSVC2019,Windows,Windows_10_22H2,1681287749,qt/qtwebengine/a2ae7142f25ce37d8df09ae50e4c7544ea8840af/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-a44c48DebugAndRelease_Sccache_UseConfigure/980811f6d5bdaf97a3b5eb38694744c5a24da5f0/test_1681289796,windows-10_22h2-msvc2019,qt/qtwebengine,Failed,5836832dfde5687dae5762509e560e1c7f2258ce,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,Integration,tst_qwebenginepage,backgroundColor,Test,Windows_10_22H2+X86_64+MSVC2019,qt/qtwebengine+tst_qwebenginepage+backgroundColor+Windows_10_22H2+X86_64+MSVC2019


In [174]:
def get_flaky_timestamp(response):
    searched_text = "FAIL!  : {}::{}".format(test_case, test_function)
    for line in response.text.split('\n'):
        if searched_text.lower() in line.lower():
            #print(line)
            timestamp = line.split(" build.go")[0]
            timestamp_str  = timestamp.split("agent:")[1]
            timestamp = line.split(" build.go")[0]
            timestamp_str  = timestamp.split("agent:")[1]
            datetime_timestamp = datetime.strptime(timestamp_str, '%Y/%m/%d %H:%M:%S')
            unix_timestamp = datetime_timestamp.replace(tzinfo=timezone.utc).timestamp()
            return(datetime_timestamp, unix_timestamp)
     

In [175]:
def get_server_id(response):
    result = re.findall(r"INFO:workitem.py:[0-9]+\([0-9]+\):.+VM metrics", response.text)
    #print(result)
    start_time = re.findall(r"from=[0-9]+", result[0])
    start_timestamp = start_time[0].split('from=')[1]
    end_time = re.findall(r"to=[0-9]+", result[0])
    end_timestamp = end_time[0].split('to=')[1]
    server_id = re.findall(r"server=[0-9]+-[0-9]+", result[0])
    server_id = server_id[0].split('server=')[1]
    #start_timestamp is in Unix seconds, you can convert it to human readable time
    start_timestamp, end_timestamp, server_id
    return(server_id)

In [176]:
test_df = pd.DataFrame(columns=columns)
columns = ['datetime_timestamp', 'unix_timestamp', 'server_id']
for index, row in df.iterrows():
    #we only need around 40 samples
    if index ==40:
        break
    url = "https://testresults.qt.io/logs/{}/log.txt.gz".format(row['path'])
    #print(url)
    response = requests.get(url)
    if response.status_code == 200:
        datetime_timestamp, unix_timestamp = get_flaky_timestamp(response)
        server_id = get_server_id(response)
        data = [[datetime_timestamp, unix_timestamp, server_id]]
        temp_df = pd.DataFrame(data,columns=columns)
        test_df = pd.concat([test_df, temp_df],ignore_index=True)
    

# getting server data from inframetrics 

In [177]:
#connecting to telegraf
telegraf_auth = json.load(open('telegraf_credentials'))
telegraf_client = InfluxDBClient(
    host="qtinfluxdb01.intra.qt.io",
    username=telegraf_auth['username'],
    password=telegraf_auth['password'],
    port=8086,
    ssl=True,
    database="coin_vms")

In [178]:
q = "show measurements"
tdf = pd.DataFrame(telegraf_client.query(q).get_points())
#tdf.head()
measurements = tdf.name.to_list()


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [179]:
for index, row in test_df.iterrows():
    dir_name = row['datetime_timestamp']
    dir_name = dir_name.strftime(("%Y_%m_%d_%H:%M:%S"))
    if os.path.exists(os.path.join(".","data", "failed", dir_name)):
        continue
    os.mkdir(os.path.join(".","data", "failed", dir_name))
    server_id = row['server_id']
    unix_timestamp = row['unix_timestamp']
    start_unix = int(unix_timestamp - 60)
    end_unix = int(unix_timestamp + 60)
    for m in measurements:
        q = "SELECT *  FROM {} WHERE host =~ /{}(-|$)/ AND time >= {}s and time <= {}s".format(m, server_id,start_unix, end_unix)
        tdf = pd.DataFrame(telegraf_client.query(q).get_points())
        if tdf.shape[0] > 0:
            tdf.to_csv(os.path.join(".","data", "failed", dir_name, m + ".csv"), index = False)
        
    
    
    

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

...

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

# getting passed data

In [187]:
q = "select * from workitem where branch = 'dev' and project = '{}' and target_os_version='{}' and target_arch='{}' and target_compiler='{}' \
and type = 'Test' and state = 'Done' and time > now() - 20d".format(project, target_os_version, target_arch, target_compiler)
print(q)
passed_df = pd.DataFrame(coin_client.query(q).get_points())
print(passed_df.shape)
passed_df.head()

select * from workitem where branch = 'dev' and project = 'qt/qtwebengine' and target_os_version='Windows_10_22H2' and target_arch='X86_64' and target_compiler='MSVC2019' and type = 'Test' and state = 'Done' and time > now() - 20d
(147, 23)


,time,branch,computeHost,cpu_count,cpu_count_1,cpu_seconds,duration,host_arch,host_compiler,host_os,host_os_version,id,path,platform_id,project,sha1,state,target_arch,target_compiler,target_os,target_os_version,taskType,type
0,2022-12-31T03:27:25Z,dev,divine-oyster.on1.qt.io,NaN,4,NaN,1768,X86_64,MSVC2019,Windows,Windows_10_22H2,nightly1681031268,qt/qtwebengine/f102d2b43f0e58ad137f1110487e2a7c437f5466/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-f9cf6aDebugAndRelease_Sccache_UseConfigure/de48ff04721dc9d7c2b8962de498448248ee8c0e/test_1681034078,None,qt/qtwebengine,cc5ec3fdf902c069a3e790638f4ee82646e30e75,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,HealthCheck,Test
1,2022-12-31T03:47:01Z,dev,star-ape.on1.qt.io,NaN,4,NaN,1599,X86_64,MSVC2019,Windows,Windows_10_22H2,nightly1681031268,qt/qtwebengine/f102d2b43f0e58ad137f1110487e2a7c437f5466/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-f9cf6aConan_DebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/377d80cd24e5ce6430ec20729cdb6eda6a07df23/test_1681034094,Windows10_22H2-MSVC2019-x64-host,qt/qtwebengine,cc5ec3fdf902c069a3e790638f4ee82646e30e75,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,HealthCheck,Test
2,2023-01-01T03:04:12Z,dev,square-tuna.on1.qt.io,NaN,4,NaN,1453,X86_64,MSVC2019,Windows,Windows_10_22H2,nightly1681036566,qt/qtwebengine/f102d2b43f0e58ad137f1110487e2a7c437f5466/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-f9cf6aDebugAndRelease_Sccache_UseConfigure/de48ff04721dc9d7c2b8962de498448248ee8c0e/test_1681039376,None,qt/qtwebengine,cc5ec3fdf902c069a3e790638f4ee82646e30e75,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,HealthCheck,Test
3,2023-01-01T03:18:02Z,dev,big-grouse.on1.qt.io,NaN,4,NaN,1648,X86_64,MSVC2019,Windows,Windows_10_22H2,nightly1681036566,qt/qtwebengine/f102d2b43f0e58ad137f1110487e2a7c437f5466/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-f9cf6aConan_DebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/377d80cd24e5ce6430ec20729cdb6eda6a07df23/test_1681039392,Windows10_22H2-MSVC2019-x64-host,qt/qtwebengine,cc5ec3fdf902c069a3e790638f4ee82646e30e75,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,HealthCheck,Test
4,2023-01-01T12:44:31Z,dev,ready-koi.on1.qt.io,NaN,4,NaN,1610,X86_64,MSVC2019,Windows,Windows_10_22H2,web_qt_qt5_1672588639366,qt/qtwebengine/f102d2b43f0e58ad137f1110487e2a7c437f5466/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64MSVC2019qtci-windows-10_22H2-x86_64-50-2b9c51DebugAndRelease_Sccache_UseConfigure/7346de38b0dcc4c14960a009b925c4f402b24986/test_1681045967,None,qt/qtwebengine,cc5ec3fdf902c069a3e790638f4ee82646e30e75,Done,X86_64,MSVC2019,Windows,Windows_10_22H2,EarlyWarning,Test


In [188]:
#exclude workitems that were flaky
df_to_exclude = df[df['full_test_params'] == checked_test_case]
ids_to_exlude = df_to_exclude.id.unique()
good_workitems_df = passed_df[~passed_df['id'].isin(ids_to_exlude)]
checked_test_case, df_to_exclude.shape, good_workitems_df.shape

('qt/qtwebengine+tst_qwebenginepage+backgroundColor+Windows_10_22H2+X86_64+MSVC2019',
 (79, 28),
 (71, 23))

In [189]:
def get_passed_timestamp(response):
    searched_text = "PASS   : {}::{}".format(test_case, test_function)
    for line in response.text.split('\n'):
        if searched_text.lower() in line.lower():
           # print(line)
            timestamp = line.split(" build.go")[0]
            timestamp_str  = timestamp.split("agent:")[1]
            timestamp = line.split(" build.go")[0]
            timestamp_str  = timestamp.split("agent:")[1]
            datetime_timestamp = datetime.strptime(timestamp_str, '%Y/%m/%d %H:%M:%S')
            unix_timestamp = datetime_timestamp.replace(tzinfo=timezone.utc).timestamp()
            return(datetime_timestamp, unix_timestamp)
     

In [184]:
good_workitems_df.shape

(3, 23)

In [190]:
columns = ['datetime_timestamp', 'unix_timestamp', 'server_id']
passed_test_df = pd.DataFrame(columns=columns)
for index, row in good_workitems_df.iterrows():
    url = "https://testresults.qt.io/logs/{}/log.txt.gz".format(row['path'])
    #print(url)
    response = requests.get(url)
    if response.status_code == 200:
        datetime_timestamp, unix_timestamp = get_passed_timestamp(response)
        server_id = get_server_id(response)
        #print(datetime_timestamp, unix_timestamp, server_id)
        data = [[datetime_timestamp, unix_timestamp, server_id]]
        temp_df = pd.DataFrame(data,columns=columns)
        passed_test_df = pd.concat([passed_test_df, temp_df],ignore_index=True)

In [191]:
passed_test_df.shape

(71, 3)

In [ ]:
for index, row in passed_test_df.iterrows():
    if index == 40:
        break
    dir_name = row['datetime_timestamp']
    dir_name = dir_name.strftime(("%Y_%m_%d_%H:%M:%S"))
    if os.path.exists(os.path.join(".", "data", "passed",  dir_name)):
        continue
    os.mkdir(os.path.join(".", "data", "passed",  dir_name))
    server_id = row['server_id']
    unix_timestamp = row['unix_timestamp']
    start_unix = int(unix_timestamp - 60)
    end_unix = int(unix_timestamp + 60)
    for m in measurements:
        q = "SELECT *  FROM {} WHERE host =~ /{}(-|$)/ AND time >= {}s and time <= {}s".format(m, server_id,start_unix, end_unix)
        tdf = pd.DataFrame(telegraf_client.query(q).get_points())
        if tdf.shape[0] > 0:
            tdf.to_csv(os.path.join('.','data', 'passed', dir_name, m + ".csv"), index = False)
        

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

...

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: Inse